# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import pickle
import re

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

from sklearn.metrics import precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from joblib import parallel_backend

from imblearn.ensemble import BalancedRandomForestClassifier

import warnings
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yvesd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yvesd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yvesd\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine(r'sqlite:///data/DisasterResponse.db', pool_pre_ping=True)
df = pd.read_sql_table('CleanData', engine)
X = df.message
Y = df[df.columns[4:]]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    """
    Normalize and tokenize message strings.
    
    Args:
    text: String - message text to process
    Returns:
    clean_tokens: list of strings - list of tokens from the message
    """
    # normalize case and remove punctuation
    text = text = re.sub('\W', ' ', text.lower())
    
    tokens = word_tokenize(text)
    stop_words = stopwords.words("english")
    
    # Reduce words to their stems
    clean_tokens = [PorterStemmer().stem(tok).strip() for tok in tokens if tok not in stop_words]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
               ('vect', CountVectorizer(tokenizer=tokenize) ),
               ('tfidf', TfidfTransformer() ),
               ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)) )
                ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
def report_results(Y_test, Y_pred):
    """Report precision, recall  and f1_score for the Machine Learning Model."""
        
    results = pd.DataFrame(columns= ['category', 'precision', 'recall', 'f1-score'])
        
    for i, category in enumerate(Y_test.columns):
        y_true = Y_test.iloc[:,i].values
        y_pred = Y_pred[:,i]
        
        row = {'category':category, 
               'precision':precision_score(y_true, y_pred, zero_division=0, average='macro'), 
               'recall':recall_score(y_true, y_pred, zero_division=0, average='macro'), 
               'f1-score':f1_score(y_true, y_pred, zero_division=0, average='macro')}
        results = results.append(row, ignore_index=True)
    
    median_values = {'category':'median_values', 
               'precision':results['precision'].median(), 
               'recall':results['recall'].median(), 
               'f1-score':results['f1-score'].median()}
    results = results.append(median_values, ignore_index=True)
    
    return results

In [7]:
pipeline.fit(X_train, Y_train)

Y_pred = pipeline.predict(X_test)

In [ ]:
print('Writing results to DB in table "Pipeline".')
report_results(Y_test, Y_pred).to_sql('Pipeline', engine, index=False, if_exists='replace')

Due to remote execution of this code we will later transfer this notebook into a plain python script and write the performance results into the existing SQL database. When we transfer the data back to our local machine we're able to read out the tables and copmpare the models.

### 6. Improve your model
Use grid search to find better parameters. 

In [24]:
def f1_scorer(y_true, y_pred):
    """
    Calculate median F1-Score to measure model performance.
    Args:
    y_true: DataFrame containing the actual labels
    y_pred: Array containing the predicted labels
    Returns:
    f1_score: Float representing the median F1-Score for the model.
    """
    scores = []
        
    for i in range(y_pred.shape[1]):
        scores.append(f1_score(np.array(y_true)[:,i], y_pred[:,i], zero_division=0, average='macro'))
    score = np.median(scores)
    return score

In [25]:
parameters = {
    'vect__ngram_range': [(1,1), (1,2), (1,4)],
    'clf__estimator__min_samples_leaf':[1, 5],
    'clf__estimator__class_weight': [None, 'balanced'],
    'clf__estimator__n_estimators': [50, 100, 200]         
    }
scorer = make_scorer(f1_scorer)
cv = GridSearchCV(pipeline, param_grid=parameters, scoring=scorer, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [28]:
cv.fit(X_train, Y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] clf__estimator__min_samples_leaf=1, vect__ngram_range=(1, 1) ....
[CV] clf__estimator__min_samples_leaf=1, vect__ngram_range=(1, 1) ....
[CV] clf__estimator__min_samples_leaf=1, vect__ngram_range=(1, 1) ....
[CV] clf__estimator__min_samples_leaf=1, vect__ngram_range=(1, 1) ....
[CV] clf__estimator__min_samples_leaf=1, vect__ngram_range=(1, 1) ....
[CV]  clf__estimator__min_samples_leaf=1, vect__ngram_range=(1, 1), score=nan, total=   0.1s
[CV]  clf__estimator__min_samples_leaf=1, vect__ngram_range=(1, 1), score=nan, total=   0.1s
[CV]  clf__estimator__min_samples_leaf=1, vect__ngram_range=(1, 1), score=nan, total=   0.1s


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s


[CV]  clf__estimator__min_samples_leaf=1, vect__ngram_range=(1, 1), score=0.548, total= 5.0min
[CV]  clf__estimator__min_samples_leaf=1, vect__ngram_range=(1, 1), score=0.541, total= 5.1min


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  5.1min finished


In [65]:
# Get results of grid search
data = {'parameter': list(cv.best_params_.keys()),
        'value': [str(value) for value in cv.best_params_.values()]}
cv_results = pd.DataFrame(data)
cv_results = cv_results.append(
    {'parameter': 'median f1-score','value': np.max(cv.cv_results_['mean_test_score'])},
    ignore_index=True)
print('Writing results of GridSearch.fit to DB in table "GsFit".')
cv_results.to_sql('GsFit', engine, index=False, if_exists='replace')

In [ ]:
Y_pred = cv.predict(X_test)
print('Writing results of GridSearch.predict to DB in table "GsPredict".')
report_results(Y_test, Y_pred).to_sql('GsPredict', engine, index=False, if_exists='replace')

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
balanced_pipeline = Pipeline([
               ('vect', CountVectorizer(tokenizer=tokenize) ),
               ('tfidf', TfidfTransformer() ),
               ('clf', MultiOutputClassifier(BalancedRandomForestClassifier(n_jobs=-1) ))
                ])

In [ ]:
keys = ['vect__ngram_range',
        'clf__estimator__min_samples_leaf',
        'clf__estimator__class_weight',
        'clf__estimator__n_estimators']
values = [cv.get_params(True)[key] for key in keys]
tuning_params = dict(zip(keys, values))

In [ ]:
balanced_pipeline.set_params(
    vect__ngram_range = tuning_params['vect__ngram_range'],
    clf__estimator__min_samples_leaf = tuning_params['clf__estimator__min_samples_leaf'],
    clf__estimator__class_weight = tuning_params['clf__estimator__class_weight'],
    clf__estimator__n_estimators = tuning_params['clf__estimator__n_estimators']
)

In [ ]:
balanced_pipeline.fit(X_train, Y_train)
Y_pred = balanced_pipeline.predict(X_test)

In [ ]:
print('Writing results of BalancedPipeline to DB in table "BalancedPipeline".')
report_results(Y_test, Y_pred).to_sql('BalancedPipeline', engine, index=False, if_exists='replace')

### 9. Export your model as a pickle file

In [ ]:
print('Saving models in pickle files.')
pickle.dump(cv, open('disaster_model.pkl', 'wb'))
pickle.dump(balanced_pipeline, open('balanced_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.